In [1]:
import os

In [2]:
pwd

'c:\\Users\\junu\\OneDrive\\Desktop\\ds\\MLOPS\\media_campign_cost_prediction\\branches\\basic-set-up\\Media-Campign-cost-prediction\\research'

In [3]:
os.chdir('../')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    data_transformation_root: Path
    data_path: Path
    transformed_data_path: Path
    target: str

In [10]:
from src.constants import *
from src.utils.common import read_yaml,create_directories

class Configuration:
    def __init__(self,config_path=CONFIG_FILE_PATH
                      ,params_path=PARAMS_FILE_PATH
                      ,schema_path=SCHEMA_FILE_PATH) -> None:
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)
        self.schema=read_yaml(schema_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation
        data_txm_config=DataTransformationConfig(
            data_transformation_root=config.data_transformation_root,
            data_path=config.data_path,
            transformed_data_path=config.transformed_data_path,
            target=config.target
        )

        return data_txm_config

        


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from src.excption.exception import customexception
import sys
class DataTransformation:
    def __init__(self,data_txm_config:DataTransformationConfig) -> None:
        self.config=data_txm_config

    def get_basic_transfomed_data(self,data: pd.DataFrame):
        if 'id' in data.columns:
            data=data.drop('id',axis=1)
        data=data.dropna()
        data=data.drop_duplicates()
        return data

        

    def data_transformation_for_training(self):
        try:
            data=pd.read_csv(self.config.data_path)
            data=self.get_basic_transfomed_data(data)
            target=self.config.target
            X=data.drop(target,axis=1)
            y=data[target]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            return X_train, X_test, y_train, y_test
        except Exception as e:
            raise customexception(e,sys)

    
    def data_transforamtion_for_prediction(self,data :pd.DataFrame):
        data=self.get_basic_transfomed_data(data)
        return data








In [18]:
class DataTransformationPipeline:
    def __init__(self) -> None:
        pass

    def main(self):
        config=Configuration()
        data_txm_config=config.get_data_transformation_config()
        data_txm=DataTransformation(data_txm_config=data_txm_config)
        X_train, X_test, y_train, y_test=data_txm.data_transformation_for_training()
        print(X_train.shape)
        
         

In [19]:
pipl=DataTransformationPipeline()
pipl.main()

[2024-03-31 14:43:37,840: INFO: 17 : call: file loaded succesfully from the path config\config.yaml]
[2024-03-31 14:43:37,843: INFO: 17 : call: file loaded succesfully from the path params.yaml]
[2024-03-31 14:43:37,856: INFO: 17 : call: file loaded succesfully from the path schema.yaml]
(252235, 15)
